<a href="https://colab.research.google.com/github/stepkurniawan/network-analysis-wikipedia-journals/blob/main/codes/2_Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import itertools

In [3]:
# run this in colab 
! git clone https://github.com/stepkurniawan/network-analysis-wikipedia-journals.git
df_journal = pd.read_csv('/content/network-analysis-wikipedia-journals/df_journal.csv')

Cloning into 'network-analysis-wikipedia-journals'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 92 (delta 8), reused 17 (delta 6), pack-reused 71
Unpacking objects: 100% (92/92), done.


FileNotFoundError: ignored

In [2]:
df_journal = pd.read_csv('df_journal.csv')

FileNotFoundError: ignored

In [ ]:
df_journal.columns

In [ ]:
g = nx.Graph()

In [ ]:
def draw_network(graph, seed=42):
    nx.draw(graph, pos=nx.spring_layout(graph, seed=seed),
            with_labels=True, node_size=1000)

In [ ]:
g.add_edges_from([['Stephen', 'Jenya'], ['Stephen', 'Beka'], ['Beka', 'Jenya']])
g.add_edges_from([['Stephen', 'Jenya'], ['Stephen', 'Jenya']])
g.add_nodes_from(['A'])
#, 'Jenya'], ['Stephen', 'Beka'], ['Beka', 'Jenya']])
draw_network(g)

In [ ]:
df_journal.nunique()

In [ ]:
df_sample_journal_data = pd.DataFrame(np.array([['p1', 'j1'],['p1', 'j2'], ['p1', 'j3'], ['p2', 'j1'], ['p2', 'j2']]),
                   columns=['page_title', 'journal_name'])
df_sample_journal_data

In [ ]:
# df_groupby = df_sample_journal_data.groupby('page_title')['journal_name'].apply(lambda x: ', '.join(x))
df_groupby = df_sample_journal_data.groupby('page_title')['journal_name'].apply(list)

df_groupby

In [ ]:
iterations_sample = itertools.combinations(df_groupby['p1'], 2)
print(iterations_sample)
for i in iterations_sample:
    print(i)

In [ ]:
df_groupby = df_journal.groupby('page_title')['journal_name'].apply(list)
df_groupby

In [ ]:
# find combination of all the journal name

iterations = itertools.combinations(df_groupby[2], 2)

for combi in iterations: 
    print(combi)

In [ ]:
# make list iterations based on the combinations

journal_relation_list = []

for page in range(len(df_groupby)):
    iterations = itertools.combinations(df_groupby[page], 2)
    for iter in iterations:
        journal_relation_list.append(iter)

#journal_relation_list


In [ ]:
df1 = pd.DataFrame (journal_relation_list, columns = ['journal1', 'journal2'])

df1

In [ ]:
df1['journal1'] = df1['journal1'].astype(str)
df1['journal2'] = df1['journal2'].astype(str)
df1

In [ ]:
df2 = df1[['journal1','journal2']] = pd.DataFrame(np.sort(df1[['journal1','journal2']].values), 
                                 index=df1.index, columns=['journal1','journal2'])
df2

In [ ]:
df3 = df2[['journal1', 'journal2']].value_counts().reset_index(name='weight')
df3.head(20)

In [ ]:
df3 = df3[df3.weight>=200]
df3

In [ ]:
df3.nunique()

In [ ]:
df3.to_csv(r'df3_weights.csv')

In [ ]:
df3.hist(column='weight')

In [ ]:
G = nx.from_pandas_edgelist(df3, 'journal1', 'journal2', 'weight')

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(10, 8))
nx.draw_shell(G, with_labels=True)

In [ ]:
def minimal_pathfinder(G, r = float("inf")):
    """ 
    Args:
    -----
    G [networkX graph]:
        Graph to filter links from.
    r [float]:
        "r" parameter as in the paper.

    Returns:
    -----
    PFNET [networkX graph]:
        Graph containing only the PFNET links.
    """
    
    import networkx as nx
    from collections import defaultdict
    
    H = G.copy()
    
    # Initialize adjacency matrix W
    W = defaultdict(lambda: defaultdict(lambda: float("inf")))
    
    # Set diagonal to 0
    for u in H.nodes():
        W[u][u] = 0 
    
    # Get weights and set W values
    for i, j, d in H.edges(data=True):
        W[i][j] = d['weight'] # Add weights to W
        
    # Get shortest path distance matrix D
    dist = nx.floyd_warshall_predecessor_and_distance(H, weight='weight')[1]
    
    # Iterate over all triples to get values for D
    for k in H.nodes():
        for i in H.nodes():
            for j in H.nodes():
                if r == float("inf"): # adapted from the R-comato version which does a similar check
                # Discard non-shortest paths
                    dist[i][j] = min(dist[i][j], (dist[i][k] + dist[k][j]))
                else:
                    dist[i][j] = min(dist[i][j], (((dist[i][k]) ** r) + ((dist[k][j]) ** r )) ** (1/r))
                
    # Check for type; set placeholder for either case
    if not H.is_directed():
        PFNET = nx.Graph()
        PFNET.add_nodes_from(H.nodes(data=True))
    else:
        PFNET = nx.DiGraph()
        PFNET.add_nodes_from(H.nodes(data=True))
        
    # Add links D_ij only if == W_ij
    for i in H.nodes():
        for j in H.nodes():
            if dist[i][j] == W[i][j]: # If shortest path distance equals distance in adjacency
                if dist[i][j] == float("inf"): # Skip infinite path lengths
                    pass
                elif i == j: # Skip the diagonal
                    pass
                else: # Add link to PFNET
                    weight = dist[i][j]
                    PFNET.add_edge(i, j, weight=weight)
                    
    return PFNET

In [ ]:
df4=df3
g = nx.from_pandas_edgelist(df4, 'journal1', 'journal2', 'weight')

In [ ]:
draw_network(g)

In [ ]:
PFNET = minimal_pathfinder(g)
print(nx.info(g))
print(nx.info(PFNET))

In [ ]:
draw_network(PFNET)

In [ ]:
nx.write_gexf(PFNET, "PFNET.gexf")

In [ ]:
df4

In [ ]:
G = nx.complete_graph(50) # Generate a complete graph

# Add random weights
for (u,v,w) in G.edges(data=True):
    w['Weight'] = np.random.randint(1,20)
    
PFNET = minimal_pathfinder(G)

print(nx.info(G))
print(nx.info(PFNET))

In [ ]:
# for (u,v,w) in G.edges(data=True):
#     print (w)